In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
try: #If running in colab
    import google.colab
    IN_COLAB = True
    %tensorflow_version 2.x
except:
    IN_COLAB = False

In [3]:
import tensorflow as tf
if (not tf.__version__.startswith('2')): #Checking if tf 2.0 is installed
    print('Please install tensorflow 2.0 to run this notebook')

print('Tensorflow version: ',tf.__version__, ' running in colab?: ', IN_COLAB)

Tensorflow version:  2.5.0  running in colab?:  False


In [4]:
# load required libraries:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.metrics import confusion_matrix

import tensorflow.keras as keras

In [5]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from extra_keras_datasets import emnist

(x_train, y_train), (x_test, y_test) = emnist.load_data(type='balanced')
# separate x_train in X_train and X_val, same for y_train

# Vorbereiten von Nist

# Sortieren Sie alle Ziffern aus, die unerwünscht sind!
# Hinweis: np.where ist hierfür recht Hilfreich
x_train = x_train[np.where(y_train < 16)]
y_train = y_train[np.where(y_train < 16)]
x_test = x_test[np.where(y_test < 16)]
y_test = y_test[np.where(y_test < 16)]

print("x_train ", x_train.shape)
print("y_train ", y_train.shape)
print("x_test ", x_test.shape)
print("y_Test ", y_test.shape)

INFO:root:Loading dataset = emnist


x_train  (38400, 28, 28)
y_train  (38400,)
x_test  (6400, 28, 28)
y_Test  (6400,)


In [6]:
X_train=x_train[0:50000] / 255 #divide by 255 so that they are in range 0 to 1
Y_train=keras.utils.to_categorical(y_train[0:50000], 16) # one-hot encoding

X_val=x_train[50000:60000] / 255
Y_val=keras.utils.to_categorical(y_train[50000:60000], 16)

X_test=x_test / 255
Y_test=keras.utils.to_categorical(y_test, 16)

del x_train, y_train, x_test, y_test

X_train=np.reshape(X_train, (X_train.shape[0],28,28,1))
X_val=np.reshape(X_val, (X_val.shape[0],28,28,1))
X_test=np.reshape(X_test, (X_test.shape[0],28,28,1))

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)

(38400, 28, 28, 1)
(0, 28, 28, 1)
(6400, 28, 28, 1)
(38400, 16)
(0, 16)
(6400, 16)


In [ ]:
# Plotten Sie hier ein paar Ziffern!
plt.figure(figsize=(432, 288))
for i in range(0, 9):
    plt.subplot(3, 3, (i + 1))
    plt.imshow((X_train[i, :, :]), cmap="gray")
    plt.title('true label: ' + np.str(np.argmax(Y_train[i])))
plt.savefig("ziffer_examples.png")

In [ ]:
# prepare data for fcNN - we need a vector as input

X_train_flat = X_train.reshape([X_train.shape[0], 784])
X_val_flat = X_val.reshape([X_val.shape[0], 784])
X_test_flat = X_test.reshape([X_test.shape[0], 784])

# check the shape
print(X_train_flat.shape)
print(Y_train.shape)
print(X_val_flat.shape)
print(Y_val.shape)

In [ ]:
base_model = tf.keras.models.load_model("cnn")

In [ ]:
#Hier die letzte Schicht entfernen
base_model.pop()
base_model.summary()

In [ ]:
# Modell einfrieren
base_model.trainable = False


In [ ]:
# Basismodell als Ausgangspunkt nehmen
inputs = tf.keras.Input(shape=(28, 28,1))
x = base_model(inputs, training=False)

In [ ]:
# Hier einen Dense-layer und einmal Softmax hinzufügen, sodass die verschiedenen Klassen vorhergesagt werden! können!
last_layer =  tf.keras.layers.Dense(16)
output     = tf.keras.layers.Activation('softmax')(last_layer)

In [ ]:
# Modell Zusammenfügen
model = tf.keras.Model(inputs, output)

In [ ]:
# Modell Kompilieren, Kostenfunktion categorical_crossentropy, Optimizer Adam mit der learningrate 1e-4

model.summary()

In [ ]:
# Accuracy vor dem Transferlearning bestimmen
loss0, accuracy0 = model.evaluate(x = X_test,y = Y_test)

In [ ]:
# Hier das Netzwerk für 10 Epochen trainieren!
# compile model and intitialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# train the model
history=model.fit(X_train_flat, Y_train,
                  batch_size=128,
                  epochs=10,
                  verbose=2,
                  validation_data=(X_val_flat, Y_val)
                 )


In [ ]:
# plot the development of the accuracy and loss during training
plt.figure(figsize=(12,4))
plt.subplot(1,2,(1))
plt.plot(history.history['accuracy'],linestyle='-.')
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.subplot(1,2,(2))
plt.plot(history.history['loss'],linestyle='-.')
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')


In [ ]:
pred=model.predict(X_test_flat)
print(confusion_matrix(np.argmax(Y_test,axis=1),np.argmax(pred,axis=1)))
acc_fc_orig = np.sum(np.argmax(Y_test,axis=1)==np.argmax(pred,axis=1))/len(pred)
print("Acc_fc_orig_flat = " , acc_fc_orig)